<a href="https://colab.research.google.com/github/choimiree/team_3/blob/main/%EB%8B%A8%EC%96%B4%EB%B3%84_%EA%B0%80%EC%A4%91%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터와 모형 불러오기

In [1]:
import pandas as pd
df = pd.read_csv('https://github.com/euphoris/datasets/raw/master/imdb.zip')

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
cv = CountVectorizer(max_features=1000, stop_words='english')

In [4]:
tdm = cv.fit_transform(df['review'])

In [5]:
x=tdm
y=df['sentiment']

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
import joblib
joblib.dump((cv, x_train, x_test, y_train, y_test), 'imdb.pkl')

['imdb.pkl']

In [8]:
cv, *_ = joblib.load('imdb.pkl')

In [9]:
import tensorflow as tf

In [10]:
model = tf.keras.models.Sequential()

In [11]:
model.add(tf.keras.layers.Dense(
    1,
    input_shape = (1000,),
    activation='sigmoid'
))

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(x_train.toarray(), y_train.values, batch_size=32, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6898 - accuracy: 0.5350
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6783 - accuracy: 0.6050
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6675 - accuracy: 0.6750
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6571 - accuracy: 0.7275
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6470 - accuracy: 0.7750
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6372 - accuracy: 0.8100
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6276 - accuracy: 0.8375
Epoch 8/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6185 - accuracy: 0.8487
Epoch 9/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6095 - accuracy: 0.8562
Epoch 10/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6009 - accuracy: 0.8662


In [14]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0)

[0.6411446928977966, 0.7599999904632568]

In [15]:
model.save('imdb-sentiment.krs')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: imdb-sentiment.krs/assets


In [16]:
model = tf.keras.models.load_model('imdb-sentiment.krs')

# 파라미터

모형의 파라미터는 각 단어에 대한 가중치(`weight`)와 y절편(`bias`)이다.

In [17]:
weight, bias = model.trainable_weights

단어와 가중치의 쌍을 표로 정리한다.

In [18]:
import pandas as pd

word_weight = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '가중치': weight.numpy().flat
})

# 부정 단어

음의 가중치를 가지는 단어가 많으면, 리뷰가 부정(0)으로 분류된다. 따라서 이들 단어는 부정 단어라고 할 수 있다.



In [19]:
word_weight.sort_values('가중치').head()


,단어,가중치
653,plot,-0.233616
67,bad,-0.206151
874,terrible,-0.204714
758,script,-0.173397
986,worst,-0.173370


나쁜(bad), 끔찍한(awful), 낭비(waste) 등이 강한 음의 가중치를 가진다.



# 긍정 단어

양의 가중치를 가지는 단어가 많으면, 리뷰가 긍정(1)으로 분류된다. 따라서 이들 단어는 긍정 단어라고 할 수 있다.



In [20]:
word_weight.sort_values('가중치').tail()


,단어,가중치
662,portrayal,0.174204
631,performance,0.182654
378,great,0.196413
74,beautiful,0.213577
976,wonderful,0.227884


좋은(fine), 유쾌한(hilarious), 흥미로운(interesting) 등이 강한 양의 가중치를 가진다.

